### Hackathon:

duration, online/offline, hackathon size, id

### Project:

team size

### People:

contributors of repositories outside the event, common event num, collaboration frequency on GitHub

1. Count contributors in a hackathon (hackathon size)

In [ ]:
"""
ALTER TABLE hackathons 
ADD COLUMN hackathon_contributors INT DEFAULT 0;
"""

"""
UPDATE hackathons h
SET hackathon_contributors = COALESCE((
    SELECT SUM(pc.team_contributor_size_during)
    FROM projects_clean pc
    WHERE pc.hackathon_url = h.url
), 0);
"""

Update hackathon & project features to projects table

In [ ]:
"""
ALTER TABLE projects
ADD COLUMN h_duration INTEGER,
ADD COLUMN is_offline_event INTEGER,
ADD COLUMN hackathon_contributor_size INTEGER,
ADD COLUMN team_contributor_size INTEGER,
ADD COLUMN hackathon_id INTEGER;

ALTER TABLE projects
ADD CONSTRAINT projects_hackathon_id_fkey 
FOREIGN KEY (hackathon_id) REFERENCES hackathons(id);

UPDATE projects
SET h_duration = EXTRACT(DAY FROM (end_date - start_date))::INTEGER
WHERE start_date IS NOT NULL AND end_date IS NOT NULL;

UPDATE projects p
SET 
    hackathon_id = h.id,
    is_offline_event = CASE 
        WHEN UPPER(TRIM(h.location)) = 'ONLINE' THEN 0 
        ELSE 1 
    END,
    hackathon_contributor_size = h.hackathon_contributors
FROM hackathons h
WHERE p.hackathon_url = h.url;

-- this contributor size is collected by contributor REST API covering contributors who contributed in any time (not only during the event)
-- team_contributor_size_during is the field we need because it only includes contributors who contributed during the event
UPDATE projects
SET team_contributor_size = ARRAY_LENGTH(
    string_to_array(
        TRIM(BOTH ',' FROM REGEXP_REPLACE(contributor_github_username, '\s+', '', 'g')),
        ','
    ),
    1
)
WHERE contributor_github_username IS NOT NULL 
  AND TRIM(contributor_github_username) != '';


SELECT 
    project_id,
    h_duration,
    is_offline_event,
    hackathon_contributor_size,
    team_contributor_size,
    hackathon_id
FROM projects
LIMIT 10;
"""

Update hackathon_participants_size in hackathons and projects

In [1]:
import pandas as pd

with open("E:\hackathon\hackathon\data\projects.csv", encoding='utf-8', errors='replace') as f:
    df = pd.read_csv(f)

if df.columns[0] == '' or df.columns[0].startswith('Unnamed'):
    df = df.iloc[:, 1:]

columns_to_keep = ['project_URL', 'submitted_to_link', 'participants_num']
df = df[columns_to_keep]

print(len(df))
print(df['participants_num'].dtype)



303680
object


In [ ]:
df['participants_num'] = pd.to_numeric(df['participants_num'], errors='coerce').astype('Int64')

print(f"Nan: {df['participants_num'].isna().sum()}")
print(f"type: {df['participants_num'].dtype}")

df.to_csv("E:\hackathon\hackathon\data\projects_temp.csv", index=False, encoding='utf-8-sig')

In [ ]:
# COPY projects_all (project_url, hackathon_url, participants_num) FROM 'E:\hackathon\hackathon\data\projects_temp.csv' WITH (FORMAT csv, HEADER true, ENCODING 'UTF8');

In [ ]:

# ALTER TABLE projects_all ADD COLUMN hackathon_id INTEGER;

# UPDATE projects_all p SET hackathon_id = h.id FROM hackathons h WHERE p.hackathon_url = h.url;

# CREATE INDEX idx_projects_all_hackathon_id ON projects_all(hackathon_id);

# ALTER TABLE projects_all ADD CONSTRAINT projects_all_hackathon_id_fkey FOREIGN KEY (hackathon_id) REFERENCES hackathons(id);

# SELECT COUNT(*) as total, COUNT(hackathon_id) as matched FROM projects_all;

In [ ]:
# ALTER TABLE hackathons ADD COLUMN hackathon_participants_size INTEGER DEFAULT 0;

# UPDATE hackathons h
# SET hackathon_participants_size = COALESCE(
#     (SELECT SUM(p.participants_num) 
#      FROM projects_all p 
#      WHERE p.hackathon_id = h.id),
#     0
# );

In [ ]:
# SELECT 
#     COUNT(*) as total_count,
#     COUNT(hackathon_participants_size) as non_null_count,
#     MIN(hackathon_participants_size) as min_value,
#     MAX(hackathon_participants_size) as max_value,
#     ROUND(AVG(hackathon_participants_size), 2) as avg_value,
#     PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY hackathon_participants_size) as q1_25th,
#     PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY hackathon_participants_size) as q2_median,
#     PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY hackathon_participants_size) as q3_75th
# FROM hackathons
# WHERE hackathon_participants_size IS NOT NULL;

In [ ]:
# ALTER TABLE projects ADD COLUMN hackathon_participants_size INTEGER;

# UPDATE projects p
# SET hackathon_participants_size = h.hackathon_participants_size
# FROM hackathons h
# WHERE p.hackathon_id = h.id;

In [ ]:
# SELECT 
#     COUNT(*) as total_count,
#     COUNT(hackathon_participants_size) as non_null_count,
#     MIN(hackathon_participants_size) as min_value,
#     MAX(hackathon_participants_size) as max_value,
#     ROUND(AVG(hackathon_participants_size), 2) as avg_value,
#     PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY hackathon_participants_size) as q1_25th,
#     PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY hackathon_participants_size) as q2_median,
#     PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY hackathon_participants_size) as q3_75th
# FROM projects
# WHERE hackathon_participants_size IS NOT NULL;

In [ ]:
# ALTER TABLE projects ADD COLUMN hackathon_size INTEGER;

# UPDATE projects SET hackathon_size = GREATEST(hackathon_participants_size, hackathon_contributor_size);

In [ ]:
# SELECT 
#     COUNT(*) as total_count,
#     COUNT(hackathon_size) as non_null_count,
#     MIN(hackathon_size) as min_value,
#     MAX(hackathon_size) as max_value,
#     ROUND(AVG(hackathon_size), 2) as avg_value,
#     PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY hackathon_size) as q1_25th,
#     PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY hackathon_size) as q2_median,
#     PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY hackathon_size) as q3_75th
# FROM projects
# WHERE hackathon_size IS NOT NULL;